#Reading and Writing DataFrames#

###DataFrame Creation### 
 **** Using Python List of tuples ****

In [0]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [0]:
df.printSchema()

In [0]:
df.show()

**createDataFrame() from SparkSession using data list**

In [0]:
dfFromData2 = spark.createDataFrame(data).toDF(*columns)

**Create DataFrame with schema**

In [0]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.printSchema()
df.show(truncate=False)

**Creating DataFrame from RDD**

**Method 1 : Using toDF() function**

In [0]:
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]
rdd = spark.sparkContext.parallelize(data)

dfFromRDD1 = rdd.toDF()

dfFromRDD1.show()

In [0]:
columns = ["language","users_count"]
dfFromRDD1 = rdd.toDF(columns)
dfFromRDD1.printSchema()

**Method 2 : Using createDataFrame() from SparkSession**
0. Using createDataFrame() from SparkSession is another way to create manually and it takes rdd object as an argument
0. chain with toDF() to specify name to the columns.

In [0]:
dfFromRDD2 = spark.createDataFrame(rdd).toDF(*columns)

**Create DataFrame from Data sources**
* csv(path)
* json(path)
* orc(path)
* parquet(path)
* table(tableName)
* text(path)
* textFile(path)
* jdbc(url, table, ..., connection properties)

In [0]:
#You can mount a Blob storage container or a folder inside a container to Databricks File System (DBFS). The mount is a pointer to a Blob storage container, so the data is never synced locally#
#the below one did not work while trying to upload csv file"
#go to data tab and upload any files)
%fs mount("E:/SampleFiles","/mnt/SampleFiles/csv")


In [0]:
%fs ls

In [0]:
csvDF = (spark.read
         .option("delimeter",",")
         .option("InferSchema","True")
         .option("header","True")
         .csv("/FileStore/tables/country_lookup.csv")
         )


In [0]:
csvDF.printSchema()

In [0]:
display(csvDF) #displays all the record in tabular format. Best way to view the output

In [0]:
csvDF.show() #shows top 20 records. Not so great for visulaization when the table contains so many columns

***spark.read.csv("path") or spark.read.format("csv").load("path")***
* including options
  * by chaining option(key,value)
  * options(Map)

* List of options available are
  * delimeter
  * inferSchema
  * header 
  * quotes
  * nullValues - sets particular string value to null
  * dateFormat
  * schema - takes schema object as an argument

In [0]:
csvDF1 = (spark.read.options(delimeter = ",", inferSchema = "True", header = "True")
         .csv("/FileStore/tables/country_lookup.csv"))

**Reading the data with User defined schema**  
Declare the Schema - list of fields and datatypes

In [0]:
# Required for StructField, StringType, IntegerType, etc.
from pyspark.sql.types import *

csvSchema = StructType([
  StructField("Country", StringType(), False),
  StructField("country_code_2_digit", StringType(), False),
  StructField("country_code_3_digit", StringType(), False),
  StructField("continent", StringType(), False),
  StructField("population", IntegerType(), False)
])

In [0]:
(spark.read                   # The DataFrameReader
  .option('header', 'true')   # Ignore line #1 - it's a header
  .option('sep', ",")        # Use tab delimiter (default is comma-separator)
  .schema(csvSchema)          # Use the specified schema
  .csv("/FileStore/tables/country_lookup.csv") # Creates a DataFrame from CSV after reading in the file
  .printSchema()
)

##Reading JSON files## 
###options###
1. read.option("multiline","true")
2. inferSchema
3. schema
4. nullValues
5. dateFormat

In [0]:
%fs head /FileStore/tables/people.json

[Truncated to first 65536 bytes]
[
{"name":"Keeley Bosco","email":"katlyn@jenkinsmaggio.net","city":"Lake Gladysberg","mac":"08:fd:0b:cd:77:f7","timestamp":"2015-04-25 13:57:36 +0700","creditcard":"1228-1221-1221-1431"},
{"name":"Rubye Jerde","email":"juvenal@johnston.name","city":null,"mac":"90:4d:fa:42:63:a2","timestamp":"2015-04-25 09:02:04 +0700","creditcard":"1228-1221-1221-1431"},
{"name":"Miss Darian Breitenberg","email":null,"city":null,"mac":"f9:0e:d3:40:cb:e9","timestamp":"2015-04-25 13:16:03 +0700","creditcard":null},
{"name":"Celine Ankunding","email":"emery_kunze@rogahn.net","city":null,"mac":"3a:af:c9:0b:5c:08","timestamp":"2015-04-25 14:22:22 +0700","creditcard":"1228-1221-1221-1431"},
{"name":"Dr. Araceli Lang","email":"mavis_lehner@jacobi.name","city":"Yvettemouth","mac":"9e:ea:28:41:2a:50","timestamp":"2015-04-25 21:02:11 +0700","creditcard":"1211-1221-1234-2201"},
{"name":"Esteban Von","email":null,"city":null,"mac":"2d:e4:f0:dd:90:96","timestamp":"2015-04-25 21:47:09 +0700","creditcard":null},
{"name":"Everette Swift","email":"gielle_jacobs@flatleyboehm.biz","city":null,"mac":"29:e0:54:7a:b7:ca","timestamp":"2015-04-25 01:42:40 +0700","creditcard":null},
{"name":"Terrell Boyle","email":"augustine.conroy@keebler.name","city":"Port Reaganfort","mac":"c5:32:09:5a:f7:15","timestamp":"2015-04-25 23:03:57 +0700","creditcard":"1228-1221-1221-1431"},
{"name":"Miss Emmie Muller","email":null,"city":"Kaleyhaven","mac":"be:dc:d2:57:81:8b","timestamp":"2015-04-25 15:48:26 +0700","creditcard":null},
{"name":"Libby Renner","email":null,"city":"Port Reneeside","mac":"9c:63:13:31:c4:ac","timestamp":"2015-04-25 08:21:59 +0700","creditcard":"1234-2121-1221-1211"},
{"name":"Kris Spencer","email":null,"city":null,"mac":"f9:8a:01:69:aa:63","timestamp":"2015-04-25 18:49:00 +0700","creditcard":null},
{"name":"Terrance Schiller","email":"prince@rosenbaum.info","city":null,"mac":"fd:b7:2a:2e:97:8f","timestamp":"2015-04-25 02:25:23 +0700","creditcard":null},
{"name":"Alessandro Barton","email":"sigurd.hudson@hodkiewicz.net","city":"South Pearl","mac":"1e:44:17:8c:c6:d8","timestamp":"2015-04-25 11:21:45 +0700","creditcard":"1234-2121-1221-1211"},
{"name":"Dr. Art Grimes","email":"monica@abbott.org","city":null,"mac":"bf:2a:a2:75:a4:38","timestamp":"2015-04-25 22:51:21 +0700","creditcard":"1211-1221-1234-2201"},
{"name":"Keven Purdy","email":"carter_zboncak@schmidtjenkins.info","city":"Port Marjolaineshire","mac":"cd:a7:57:c0:03:50","timestamp":"2015-04-25 10:13:00 +0700","creditcard":"1211-1221-1234-2201"},
{"name":"William Wisozk DVM","email":"adonis@dach.net","city":"Rogelioton","mac":"9a:6e:08:fe:8d:41","timestamp":"2015-04-25 20:57:29 +0700","creditcard":null},
{"name":"Oma Grady","email":"laurianne_goldner@kerluke.biz","city":"South Chelsie","mac":"9e:60:32:9f:88:9d","timestamp":"2015-04-25 04:08:07 +0700","creditcard":"1212-1221-1121-1234"},
{"name":"Annie Schamberger","email":"blanca_smitham@pfeffervon.net","city":"Marvinville","mac":"f0:45:37:1b:d8:19","timestamp":"2015-04-25 10:28:07 +0700","creditcard":null},
{"name":"Jazmin Kovacek","email":"mittie.ullrich@boehm.name","city":null,"mac":"13:36:6a:b4:2d:07","timestamp":"2015-04-25 19:20:51 +0700","creditcard":"1211-1221-1234-2201"},
{"name":"Thomas O'Conner","email":null,"city":"North Tierramouth","mac":"b1:cd:47:c7:ab:82","timestamp":"2015-04-25 09:58:01 +0700","creditcard":null},
{"name":"Jordi Jaskolski II","email":null,"city":null,"mac":"e8:32:67:7d:7c:b2","timestamp":"2015-04-25 03:19:14 +0700","creditcard":"1212-1221-1121-1234"},
{"name":"Willy Beahan III","email":null,"city":"Port Heloise","mac":"a1:ec:ec:06:f3:ad","timestamp":"2015-04-25 02:11:16 +0700","creditcard":"1212-1221-1121-1234"},
{"name":"Alessandra Nader Sr.","email":null,"city":"Yasminville","mac":"8e:6a:df:fb:35:4a","timestamp":"2015-04-25 05:15:08 +0700","creditcard":"1211-1221-1234-2201"},
{"name":"Leora Heller","email":null,"city":"Port Jerad","mac":"c6:bf:5a:51:c2:70","timesta

In [0]:
jsonDF = (spark.read                  
  .json("/FileStore/tables/people.json") # Creates a DataFrame from text after reading in the file
)

In [0]:
jsonDF.show()

+---------------+--------------------+-------------------+--------------------+-----------------+--------------------+--------------------+
_corrupt_record| city| creditcard| email| mac| name| timestamp|
+---------------+--------------------+-------------------+--------------------+-----------------+--------------------+--------------------+
 [| null| null| null| null| null| null|
 null| Lake Gladysberg|1228-1221-1221-1431|katlyn@jenkinsmag...|08:fd:0b:cd:77:f7| Keeley Bosco|2015-04-25 13:57:...|
 null| null|1228-1221-1221-1431|juvenal@johnston....|90:4d:fa:42:63:a2| Rubye Jerde|2015-04-25 09:02:...|
 null| null| null| null|f9:0e:d3:40:cb:e9|Miss Darian Breit...|2015-04-25 13:16:...|
 null| null|1228-1221-1221-1431|emery_kunze@rogah...|3a:af:c9:0b:5c:08| Celine Ankunding|2015-04-25 14:22:...|
 null| Yvettemouth|1211-1221-1234-2201|mavis_lehner@jaco...|9e:ea:28:41:2a:50| Dr. Araceli Lang|2015-04-25 21:02:...|
 null| null| null| null|2d:e4:f0:dd:90:96| Esteban Von|2015-04-25 21:47:...|
 null| null| null|gielle_jacobs@fla...|29:e0:54:7a:b7:ca| Everette Swift|2015-04-25 01:42:...|
 null| Port Reaganfort|1228-1221-1221-1431|augustine.conroy@...|c5:32:09:5a:f7:15| Terrell Boyle|2015-04-25 23:03:...|
 null| Kaleyhaven| null| null|be:dc:d2:57:81:8b| Miss Emmie Muller|2015-04-25 15:48:...|
 null| Port Reneeside|1234-2121-1221-1211| null|9c:63:13:31:c4:ac| Libby Renner|2015-04-25 08:21:...|
 null| null| null| null|f9:8a:01:69:aa:63| Kris Spencer|2015-04-25 18:49:...|
 null| null| null|prince@rosenbaum....|fd:b7:2a:2e:97:8f| Terrance Schiller|2015-04-25 02:25:...|
 null| South Pearl|1234-2121-1221-1211|sigurd.hudson@hod...|1e:44:17:8c:c6:d8| Alessandro Barton|2015-04-25 11:21:...|
 null| null|1211-1221-1234-2201| monica@abbott.org|bf:2a:a2:75:a4:38| Dr. Art Grimes|2015-04-25 22:51:...|
 null|Port Marjolaineshire|1211-1221-1234-2201|carter_zboncak@sc...|cd:a7:57:c0:03:50| Keven Purdy|2015-04-25 10:13:...|
 null| Rogelioton| null| adonis@dach.net|9a:6e:08:fe:8d:41| William Wisozk DVM|2015-04-25 20:57:...|
 null| South Chelsie|1212-1221-1121-1234|laurianne_goldner...|9e:60:32:9f:88:9d| Oma Grady|2015-04-25 04:08:...|
 null| Marvinville| null|blanca_smitham@pf...|f0:45:37:1b:d8:19| Annie Schamberger|2015-04-25 10:28:...|
 null| null|1211-1221-1234-2201|mittie.ullrich@bo...|13:36:6a:b4:2d:07| Jazmin Kovacek|2015-04-25 19:20:...|
+---------------+--------------------+-------------------+--------------------+-----------------+--------------------+--------------------+
only showing top 20 rows

##Reading Parquet files## 
<strong style="font-size:larger">"</strong>Apache Parquet is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language.<strong style="font-size:larger">"</strong><br>

-sandbox
### About Parquet Files
* Free & Open Source.
* Increased query performance over row-based data stores.
* Provides efficient data compression.
* Designed for performance on large data sets.
* Supports limited schema evolution.
* Is a splittable "file format".
* A <a href="https://en.wikipedia.org/wiki/Column-oriented_DBMS" target="_blank">Column-Oriented</a> data store

&nbsp;&nbsp;&nbsp;&nbsp;** Row Format ** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Column Format**

<table style="border:0">

  <tr>
    <th>ID</th><th>Name</th><th>Score</th>
    <th style="border-top:0;border-bottom:0">&nbsp;</th>
    <th>ID:</th><td>1</td><td>2</td>
    <td style="border-right: 1px solid #DDDDDD">3</td>
  </tr>

  <tr>
    <td>1</td><td>john</td><td>4.1</td>
    <td style="border-top:0;border-bottom:0">&nbsp;</td>
    <th>Name:</th><td>john</td><td>mike</td>
    <td style="border-right: 1px solid #DDDDDD">sally</td>
  </tr>

  <tr>
    <td>2</td><td>mike</td><td>3.5</td>
    <td style="border-top:0;border-bottom:0">&nbsp;</td>
    <th style="border-bottom: 1px solid #DDDDDD">Score:</th>
    <td style="border-bottom: 1px solid #DDDDDD">4.1</td>
    <td style="border-bottom: 1px solid #DDDDDD">3.5</td>
    <td style="border-bottom: 1px solid #DDDDDD; border-right: 1px solid #DDDDDD">6.4</td>
  </tr>

  <tr>
    <td style="border-bottom: 1px solid #DDDDDD">3</td>
    <td style="border-bottom: 1px solid #DDDDDD">sally</td>
    <td style="border-bottom: 1px solid #DDDDDD; border-right: 1px solid #DDDDDD">6.4</td>
  </tr>

</table>

See also
* <a href="https://parquet.apache.org/" target="_blank">https&#58;//parquet.apache.org</a>
* <a href="https://en.wikipedia.org/wiki/Apache_Parquet" target="_blank">https&#58;//en.wikipedia.org/wiki/Apache_Parquet</a>

In [0]:
parquetDF = (spark.read                  
  .parquet("/FileStore/tables/userdata1.parquet") # Creates a DataFrame from text after reading in the file
)

In [0]:
parquetDF.show()

+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
 registration_dttm| id|first_name|last_name| email|gender| ip_address| cc| country| birthdate| salary| title| comments|
+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
2016-02-03 07:55:29| 1| Amanda| Jordan| ajordan0@com.com|Female| 1.197.201.2| 6759521864920116| Indonesia| 3/8/1971| 49756.53| Internal Auditor| 1E+02|
2016-02-03 17:04:03| 2| Albert| Freeman| afreeman1@is.gd| Male| 218.111.175.34| | Canada| 1/16/1968|150280.17| Accountant IV| |
2016-02-03 01:09:31| 3| Evelyn| Morgan|emorgan2@altervis...|Female| 7.161.136.94| 6767119071901597| Russia| 2/1/1960|144972.51| Structural Engineer| |
2016-02-03 00:36:21| 4| Denise| Riley| driley3@gmpg.org|Female| 140.35.109.83| 3576031598965625| China| 4/8/1997| 90263.05|Senior Cost Accou...| |
2016-02-03 05:05:31| 5| Carlos| Burns|cburns4@miitbeian...| | 169.113.235.40| 5602256255204850| South Africa| | null| | |
2016-02-03 07:22:34| 6| Kathryn| White| kwhite5@google.com|Female| 195.131.81.179| 3583136326049310| Indonesia| 2/25/1983| 69227.11| Account Executive| |
2016-02-03 08:33:08| 7| Samuel| Holmes|sholmes6@foxnews.com| Male| 232.234.81.197| 3582641366974690| Portugal|12/18/1987| 14247.62|Senior Financial ...| |
2016-02-03 06:47:06| 8| Harry| Howell| hhowell7@eepurl.com| Male| 91.235.51.73| |Bosnia and Herzeg...| 3/1/1962|186469.43| Web Developer IV| |
2016-02-03 03:52:53| 9| Jose| Foster| jfoster8@yelp.com| Male| 132.31.53.61| | South Korea| 3/27/1992|231067.84|Software Test Eng...| 1E+02|
2016-02-03 18:29:47| 10| Emily| Stewart|estewart9@opensou...|Female| 143.28.251.245| 3574254110301671| Nigeria| 1/28/1997| 27234.28| Health Coach IV| |
2016-02-03 00:10:42| 11| Susan| Perkins| sperkinsa@patch.com|Female| 180.85.0.62| 3573823609854134| Russia| |210001.95| | |
2016-02-03 18:04:34| 12| Alice| Berry|aberryb@wikipedia...|Female| 246.225.12.189| 4917830851454417| China| 8/12/1968| 22944.53| Quality Engineer| |
2016-02-03 18:48:17| 13| Justin| Berry|jberryc@usatoday.com| Male| 157.7.146.43|6331109912871813274| Zambia| 8/15/1975| 44165.46|Structural Analys...| |
2016-02-03 21:46:52| 14| Kathy| Reynolds|kreynoldsd@redcro...|Female| 81.254.172.13| 5537178462965976|Bosnia and Herzeg...| 6/27/1970|286592.99| Librarian| |
2016-02-03 08:53:23| 15| Dorothy| Hudson|dhudsone@blogger.com|Female| 8.59.7.0| 3542586858224170| Japan|12/20/1989|157099.71| Nurse Practicioner|<script>alert('hi...|
2016-02-03 00:44:01| 16| Bruce| Willis|bwillisf@bluehost...| Male|239.182.219.189| 3573030625927601| Brazil| |239100.65| | |
2016-02-03 00:57:45| 17| Emily| Andrews|eandrewsg@cornell...|Female| 29.231.180.172| 30271790537626| Russia| 4/13/1990|116800.65| Food Chemist| |
2016-02-03 16:44:24| 18| Stephen| Wallace|swallaceh@netvibe...| Male| 152.49.213.62| 5433943468526428| Ukraine| 1/15/1978|248877.99|Account Represent...| |
2016-02-03 11:45:54| 19| Clarence| Lawson|clawsoni@vkontakt...| Male| 107.175.15.152| 3544052814080964| Russia| |177122.99| | |
2016-02-03 10:30:36| 20| Rebecca| Bell| rbellj@bandcamp.com|Female|172.215.104.127| | China| |137251.19| | |
+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
only showing top 20 rows

In [0]:
%fs head /FileStore/tables/userdata1.parquet

[Truncated to first 65536 bytes]
PAR1ȺȺL�*�l�N%^0��7N%.�"�N%���N%N��N%ļ�&N%��cN%�7N%��W�N%N{~�<N%�.z�N%t�8/;N%*��=N%�g�PGN%^rGN%���fN%�&N%PN�6N%�O�&N%�Vai"N%蟌�+N%*��;N%�+��=N%�v�gN%.[�?7N%��`UN%��VtN%0�)8N%lY�
N%�J�FN%�;��>N%���EN%�հN%��KN%�?X�N%��pN%�DN%lQ�IN%,%�0N%����N%��AN%���N%4��N%V��{!N%z<jd>N%�%luN%�v��N%�W�+N%z�"o:N%R��
N%@ |�N%���N%���QN%rO=�N%J<��4N%ʗW3N%P��N%f�lN%Z��a8N%h,&jN%�II/#N%J��EN%.q.1BN%�_R]N%�BN%8��DN%��� N%�$�N%�,6�N%���<N%.�HN%\ʤBBN%p.JD<N%(�)�N%�(@y?N%\��.N%V�a�N%����N%���-N%��^pKN%��/HN%����MN%3�r,N%�%�N%\�cn!N%F_�7BN%�J�N%��+N%�%N%�[�o=N%�A0N%ZY-N%p�@2N%�w�5N%�K!u%N%�E�&N%�O�KN%n9�jN%��.-N%��.�3N%����9N%�c�?N%�sJ?N%�o�2N%��
�7N%�z�FN%BD�KN%
�5HN%��.?N%�'.BN%����N%8�6NN%�u�K2N%���N%�g�dN%>�_N%xZ�N%X"N�BN%�x@N%����.N%�`�JN%���6.N%�h��N%���N%D��wBN%zS�88N%�NthKN%�(�!N%�D1N%t��%N%�̪�N%�B˳DN%���w	N%���N%�距4N%*��&N%�I��N%�~�	N%(�30N%��U9N%.!��N%�^V�N%�G�l;N%Z�[�<N%
���@N%r�l$0N%���N%�9�L9N%�-�N%x\��N%�&�i@N%&�?�,N%��$ZHN%���97N%::e�!N%����$N%�'t,N%X��HN%�\�#N%p��FN%<��HN%��{�N%�"N%ޭ��DN%�5��KN%V"�_<N%�c֊<N%�8�z6N%���N%����N%�?��N%|��2N%�aN�N%`!��2N%$^ʞN%C$YN%,�/JKN%��N%|fB�#N%dA�;N%�&��CN%��N%����N%DLuN%N=}NN%��YN%����N%
�LEN%Rږ�N%�t׿/N%�/��1N%�͟y6N%�h�MN%~�=8N%�؟N%��4N%n�u1N%.�L�N%L�aN%8g�1N%��a
N%�G,RN%^(ēIN%<$�Q;N%��پN%�ӚN%���>N%��@N#N%�v��1N%��,N%��!N%�j�r!N%t�\N%(eU?N%	̒N%��f�9N%��h�,N%(q�7N%dY7�;N%6��=N%��IN%�׽eN%���<0N%�R�\5N%&�i�N%��%N%!n�,N%8oMN%�I%JN%���N%VS�L8N%�v��N%����,N%8�GN%+OHN%�>i�N%���N%"ӏ"0N%�C�N%v��N%ȖK N%�L�KN%n�3�"N%�|rN%��[ N% >�):N%�wpN%
JslN%�i�9N%��y;0N%�'N%�ya�JN%Գ�N%>�N%���4N%�:1D
N%Ts�NDN%�a�0N%.�"�5N%,��.N%J���N%�[��N%��'�4N%e	�EN%��٘N%8T�QN%ꄑ�N%�8�iJN%`�EN%M+-N%�q��KN%6��N%N�1N%ޞ�N%�Ɯ�6N%���@N%�h��2N%�\��%N%����5N%�¦�+N%�B_�N%���&N%�ӵ1N%T�N%��G>,N%�l�N%�VN%D��$N%��D6N%�.�i5N%'�zN%�Ɩ�N%"2
�N%��?N%:��.N%0r'�N%XWN%`��e!N%��l6N%(� �;N%���:N%���3N% ���N%J�b:N%<=6/N%J{�MN%�3��N%ҪY�2N%�>�g6N%֯q�CN%B��l7N%��6N%��u�N%0B(3N%���	N%�_{5N%r,L�N%F.N%T�V N%b�,P8N%2��N%~4��?N%x=*�&N%R�N%���N%f��AN%�SZ
.N%�I�PDN%�r�N%�&�N%F&W/:N%d��y2N%hɥz?N%�z N%�`h=N%9lN%�Q��N%&b��9N%n+�N%�*�AN%���=4N%�}ɜ4N%=�7N%`v��N%pƅ�3N%R�@�N%����$N%���1N%h��N%ֲhLN%�sZ�"N%lU�E#N%B�߃4N%l�[1N%�!rvCN%.aBN%<*�N%
^��?N%<k0BN%B��7$N%,�}N%֧��N%�G�8N%�E��!N%V'��N%.��IDN%:e�5N%0�w�N%�N*N%P�k�N%b�OGN%�u�LN%��u_N%�ԍ�
N%$�aZFN%���N%�c9N%�А�>N%Vvv�N%J��DN%.e�SN%�iS/N%��VN%F,E�JN%p��zN%dL�IN%�}A>N%0x��N%
�L�N%2�ÜN%P-�N%v�!N%*OI�N%*��jFN%V���DN%�E=N%�z��/N%���IN%
	p�9N%n�9�N%�X$N%�}��7N%��_�*N%��S2N%����/N%@e��>N%���N%�1�7N%I(	N%�f��N%H���	N%X���CN%4�.N%�J�0N%��K�N%�l@ZKN%΀h$N%����EN%\�NXN%Fa��N%h9v*6N%���*N%Ig�NN%�AN%���YN%D�F�N%��(N%NA�EN%^��N%��| N%jry"N%�c��N%J:�N%���#N%H!b[N%Ɛ�|DN%N�Q�GN%
�՛N%J�r�N%ʷm�EN%�5}+N%���N%�73AN%d`4�:N%f��8N%zݗ@N%v�NN%pA��>N%V�7N%>X� FN%��ޔ	N%�z6N%����?N%@�1K/N%��ON%b��dAN%��~#N%c4[N%�2�N%��19N%�E�N%P�\N%��Z.NN%<΄N%�"bN%Li/�?N%�,"?N%J�HN%�?�"N%��yN%���9N%P~��N%2�pPN%6G��$N%�#a�HN%��B�
N%Ԁ�1N%N�o�N%��WG2N%v��N%�t"N%�:��N%����@N%Pɻ�+N%�(W8N%~��N%�A�BN%��.N%T��N%RM^�N%�J<�%N%
��aN%��E�0N%:ǝ�"N%0q+N%���N%z�v�N%�í<N%�F��N%$��/N%`��AN%��X?N%�j��;N%�/��N%"8�NN%�g�	-N%�_�t1N%�ȉN%�7-�@N%��HN%�OmNN%>��AN%�NnTKN%�6�>	N%��y5N%0�e'DN%�pؕ$N%r5̲N%��r�N%�<��AN%t�wSN%���N%Η��N%�+Z�N%��\^N%��'N%�d��;N%��Q/N%�%�N%Ů�HN%���N%��v	IN%�p�$N%P�J�3N%��- DN%�G��N%$�LN%�m�QGN%�ް�N%��MN%��AN%mN%�QJN%2�E:N%�5"!N%Z�mOJN%0TK�N%^��N%�l��&N%t��HN%��ʾN%���WN%���c9N%`�Z"N%��R� N%���=MN%r�N%0M�:!N%����N%�&N%f+��N%h��DN%�P�5N%�v8�>N% ި�=N%N��*N%*�Em:N%N�ڗDN%��

##Writing Dataframes to different formats## 
**df.write** 
csv

  0. Options: 
    * header
    * delimeter 
    * quote
    * escape
    * nullValues
    * dateFormat
    * quoteMode

**Saving modes**
-PySpark DataFrameWriter also has a method mode() to specify saving mode. 

* **.mode **

  overwrite – mode is used to overwrite the existing file.

  append – To add the data to the existing file.

  ignore – Ignores write operation when the file already exists.

  error – This is a default option when the file already exists, it returns an error.

In [0]:
parquetDF.write.options(header='True', delimiter=',').csv("/FileStore/tables/writeDF")

In [0]:
%fs ls "/FileStore/tables/writeDF"

path,name,size
dbfs:/FileStore/tables/writeDF/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/tables/writeDF/_committed_4317844966561036838,_committed_4317844966561036838,112
dbfs:/FileStore/tables/writeDF/_started_4317844966561036838,_started_4317844966561036838,0
dbfs:/FileStore/tables/writeDF/part-00000-tid-4317844966561036838-2610edd0-1a3d-49c5-ba86-b126ebaa364e-34-1-c000.csv,part-00000-tid-4317844966561036838-2610edd0-1a3d-49c5-ba86-b126ebaa364e-34-1-c000.csv,149366


In [0]:
%fs head /FileStore/tables/writeDF/part-00000-tid-4317844966561036838-2610edd0-1a3d-49c5-ba86-b126ebaa364e-34-1-c000.csv


[Truncated to first 65536 bytes]
registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
2016-02-03T07:55:29.000Z,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
2016-02-03T17:04:03.000Z,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,"",Canada,1/16/1968,150280.17,Accountant IV,""
2016-02-03T01:09:31.000Z,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,""
2016-02-03T00:36:21.000Z,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,""
2016-02-03T05:05:31.000Z,5,Carlos,Burns,cburns4@miitbeian.gov.cn,"",169.113.235.40,5602256255204850,South Africa,"","","",""
2016-02-03T07:22:34.000Z,6,Kathryn,White,kwhite5@google.com,Female,195.131.81.179,3583136326049310,Indonesia,2/25/1983,69227.11,Account Executive,""
2016-02-03T08:33:08.000Z,7,Samuel,Holmes,sholmes6@foxnews.com,Male,232.234.81.197,3582641366974690,Portugal,12/18/1987,14247.62,Senior Financial Analyst,""
2016-02-03T06:47:06.000Z,8,Harry,Howell,hhowell7@eepurl.com,Male,91.235.51.73,"",Bosnia and Herzegovina,3/1/1962,186469.43,Web Developer IV,""
2016-02-03T03:52:53.000Z,9,Jose,Foster,jfoster8@yelp.com,Male,132.31.53.61,"",South Korea,3/27/1992,231067.84,Software Test Engineer I,1E+02
2016-02-03T18:29:47.000Z,10,Emily,Stewart,estewart9@opensource.org,Female,143.28.251.245,3574254110301671,Nigeria,1/28/1997,27234.28,Health Coach IV,""
2016-02-03T00:10:42.000Z,11,Susan,Perkins,sperkinsa@patch.com,Female,180.85.0.62,3573823609854134,Russia,"",210001.95,"",""
2016-02-03T18:04:34.000Z,12,Alice,Berry,aberryb@wikipedia.org,Female,246.225.12.189,4917830851454417,China,8/12/1968,22944.53,Quality Engineer,""
2016-02-03T18:48:17.000Z,13,Justin,Berry,jberryc@usatoday.com,Male,157.7.146.43,6331109912871813274,Zambia,8/15/1975,44165.46,Structural Analysis Engineer,""
2016-02-03T21:46:52.000Z,14,Kathy,Reynolds,kreynoldsd@redcross.org,Female,81.254.172.13,5537178462965976,Bosnia and Herzegovina,6/27/1970,286592.99,Librarian,""
2016-02-03T08:53:23.000Z,15,Dorothy,Hudson,dhudsone@blogger.com,Female,8.59.7.0,3542586858224170,Japan,12/20/1989,157099.71,Nurse Practicioner,<script>alert('hi')</script>
2016-02-03T00:44:01.000Z,16,Bruce,Willis,bwillisf@bluehost.com,Male,239.182.219.189,3573030625927601,Brazil,"",239100.65,"",""
2016-02-03T00:57:45.000Z,17,Emily,Andrews,eandrewsg@cornell.edu,Female,29.231.180.172,30271790537626,Russia,4/13/1990,116800.65,Food Chemist,""
2016-02-03T16:44:24.000Z,18,Stephen,Wallace,swallaceh@netvibes.com,Male,152.49.213.62,5433943468526428,Ukraine,1/15/1978,248877.99,Account Representative I,""
2016-02-03T11:45:54.000Z,19,Clarence,Lawson,clawsoni@vkontakte.ru,Male,107.175.15.152,3544052814080964,Russia,"",177122.99,"",""
2016-02-03T10:30:36.000Z,20,Rebecca,Bell,rbellj@bandcamp.com,Female,172.215.104.127,"",China,"",137251.19,"",""
2016-02-03T13:17:24.000Z,21,Diane,Stevens,dstevensk@cnet.com,Female,141.243.73.164,"",Russia,6/5/1985,87978.22,Food Chemist,œ∑´®†¥¨ˆøπ“‘
2016-02-03T18:18:25.000Z,22,Lawrence,Ramos,lramosl@sourceforge.net,Male,46.72.4.6,3537473810855655,Tanzania,"",131283.64,"",""
2016-02-03T18:50:55.000Z,23,Gregory,Barnes,gbarnesm@google.ru,Male,220.22.114.145,3538432455620641,Tunisia,1/23/1971,182233.49,Senior Sales Associate,사회과학원 어학연구소
2016-02-03T04:05:40.000Z,24,Michelle,Ellis,mellisn@timesonline.co.uk,Female,239.81.215.135,3547383558025965,Tanzania,6/5/1964,278001.46,Tax Accountant,""
2016-02-03T16:52:27.000Z,25,Rachel,Perkins,rperkinso@lulu.com,Female,90.173.28.95,633313663891003209,Russia,"",176178.75,"",""
2016-02-03T08:02:34.000Z,26,Anthony,Lawrence,alawrencep@miitbeian.gov.cn,Male,121.211.242.99,564182969714151470,Japan,12/10/1979,170085.81,Electrical Engineer,""
2016-02-03T08:41:26.000Z,27,Henry,Henry,hhenryq@godaddy.com,Male,191.88.236.116,4905730021217853521,China,9/22/1995,284300.15

**Writing in parquet file format**
* .option - compression (none, snappy, gzip, lzo)  
* .mode - saving mode (overwrite, append, ignore, error)
* .partitionBy - while saving the file we can partition the data. Retreiving the particular partition becomes easier and improves performance.

In [0]:
jsonDF.write.mode('overwrite').option("compression","snappy").partitionBy('city').parquet("/FileStore/tables/writeDF/people.parquet")